In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
import pathlib

C:\Users\vllla\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_dir = "C:\\Users\\vllla\Downloads\\kagglecatsanddogs_5340_resize\\PetImages"

In [4]:
img_height = 32
img_width = 32

batch_size = 32

In [5]:
batch_size = 32

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  seed=123,
  subset="training",
  validation_split = 0.2,
  shuffle = True,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 24994 files belonging to 2 classes.
Using 19996 files for training.


In [7]:
class_names = train_ds.class_names

In [8]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  seed=123,
  validation_split = 0.2,
  subset="validation",
  image_size=(img_height, img_width), 
  batch_size=batch_size)

Found 24994 files belonging to 2 classes.
Using 4998 files for validation.


In [9]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)# чтобы быстрее работал
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [4]:
import os
from tqdm import tqdm

CATEGORIES = ["Dog", "Cat"]

In [5]:
CATEGORIES

['Dog', 'Cat']

In [6]:
train_ds = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(data_dir,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (img_height, img_width))  # resize to normalize data size
                train_ds.append([new_array, class_num])  # add this to our training_data
            except Exception as e: 
                pass

create_training_data()

print(len(train_ds))

100%|██████████| 12495/12495 [00:05<00:00, 2411.26it/s]

24994


In [7]:
import random

random.shuffle(train_ds)

In [8]:
X = []
y = []

for features,label in train_ds:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, img_width, img_height, 1)

In [10]:
def nn():
    model = tf.keras.Sequential([
        # tf.keras.layers.Resizing(img_height, img_width,),
        # tf.keras.layers.layers.layers.Rescaling(1./255),
        tf.keras.layers.Conv2D(16, kernel_size = (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(32, kernel_size = (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(64, kernel_size = (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(128, kernel_size = (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(128, activation='sigmoid'),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    
    return model

In [10]:
input_shape = (None, 32, 32, 3)

In [52]:
model = nn()
# model.build(input_shape)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 16)        448       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 32)        4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 4, 4, 64)        

In [53]:
lr = 0.0003
optimizer = keras.optimizers.Adam(learning_rate = lr)

In [25]:
callback = tf.keras.callbacks.ModelCheckpoint('cifar_weight.h5', save_weights_only = True)

In [54]:
model.compile(
  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer = optimizer,
  metrics = [keras.metrics.SparseCategoricalAccuracy()])

In [55]:
from tensorflow.keras.callbacks import TensorBoard

In [56]:
tensorboard = TensorBoard(log_dir = 'logs/model')

In [57]:
model.fit(
  x = train_ds,
   batch_size=32,
  epochs=9,
  validation_data = val_ds,
  callbacks = [tensorboard],
)

Epoch 1/9


C:\Users\vllla\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


625/625 [==============================] - 20s 31ms/step - loss: 0.7444 - sparse_categorical_accuracy: 0.5029 - val_loss: 0.6946 - val_sparse_categorical_accuracy: 0.5114
Epoch 2/9
625/625 [==============================] - 10s 16ms/step - loss: 0.7268 - sparse_categorical_accuracy: 0.5004 - val_loss: 0.6936 - val_sparse_categorical_accuracy: 0.5114
Epoch 3/9
625/625 [==============================] - 9s 14ms/step - loss: 0.7136 - sparse_categorical_accuracy: 0.5010 - val_loss: 0.6955 - val_sparse_categorical_accuracy: 0.4886
Epoch 4/9
625/625 [==============================] - 8s 12ms/step - loss: 0.7068 - sparse_categorical_accuracy: 0.4996 - val_loss: 0.6958 - val_sparse_categorical_accuracy: 0.5114
Epoch 5/9
625/625 [==============================] - 9s 15ms/step - loss: 0.7028 - sparse_categorical_accuracy: 0.5000 - val_loss: 0.7092 - val_sparse_categorical_accuracy: 0.4886
Epoch 6/9
625/625 [==============================] - 9s 14ms/step - loss: 0.6996 - sparse_categorical_accura

In [76]:
model.save('CNN_cat_vs_dog.model')

INFO:tensorflow:Assets written to: CNN_cat_vs_dog.model\assets


INFO:tensorflow:Assets written to: CNN_cat_vs_dog.model\assets


In [71]:
model.load_weights('cifar_weight.h5')

In [77]:
model = tf.keras.models.load_model("CNN_cat_vs_dog.model")

In [78]:
# "C:\Users\vllla\Downloads\kagglecatsanddogs_5340\PetImages\test\Cat\1005.jpg"

img = tf.keras.utils.load_img(
    r"C:\Users\vllla\Downloads\shutterstock_211592482-e1528481509952.jpg",
    target_size=(img_height, img_width)
)
# image = tf.keras.preprocessing.image.load_img(r"C:\Users\vllla\Downloads\kagglecatsanddogs_5340\PetImages\Dog\1425.jpg")
input_arr = tf.keras.preprocessing.image.img_to_array(img)
input_arr = np.array([input_arr])  # Convert single image to a batch.
predictions = model.predict(input_arr)
predictions[0]

1/1 [==============================] - 0s 106ms/step


array([0.00453295, 0.99546707], dtype=float32)

In [64]:
path =  r"C:\Users\vllla\Downloads\kagglecatsanddogs_5340\PetImages\Cat\43.jpg"


img_array = cv2.imread(os.path.join(path), cv2.IMREAD_UNCHANGED)  # convert to array
new_array = cv2.resize(img_array, (img_height, img_width))  # resize to normalize data size
new_array = np.array(new_array).reshape(-1, img_width, img_height, 3)
predictions = model.predict(new_array)
predictions[0]

1/1 [==============================] - 0s 20ms/step


array([0.5162315 , 0.48376855], dtype=float32)

In [40]:
predictions[0]

array([0.9965599 , 0.00344004], dtype=float32)

In [75]:
i=np.where(predictions[0] == max(predictions[0]))[0][0]
print(predictions[0][i])
class_names[i]

0.99546707


'Dog'

In [38]:
print(f"This image most likely belongs to {labels[i]} with a {round(100 * predictions[0][i], 2)}%.") 

This image most likely belongs to Cat with a 68.16%.


In [37]:
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

In [71]:
predictions = model.predict_generator(img_array)
score = np.argmax(predictions,axis=1)

C:\Users\vllla\AppData\Local\Temp\ipykernel_668\1616381710.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(img_array)


In [75]:
labels = (train_ds)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [labels[k] for k in score]

AttributeError: 'PrefetchDataset' object has no attribute 'class_indices'

In [73]:
labels

['Cat', 'Dog']

In [ ]:
# filenames=val_ds.filenames
# results=pd.DataFrame({"Filename":filenames,
#                       "Predictions":predictions})

In [39]:
class_names = train_ds.class_indices

AttributeError: 'BatchDataset' object has no attribute 'class_indices'

In [20]:
with open('weight.txt', 'w') as f:
    for i in class_names:
        f.write(f'{i} {class_names[i]}\n')

In [69]:
print(score[0][0].numpy())

0.47664538


In [70]:
if score[0][0].numpy() > 0.5:
    print('cat', score[0][0].numpy())
else:
    print( 'dog', score[0][1].numpy())

dog 0.52335465


In [60]:
print(f"This image most likely belongs to  with a {round(100 * score[0][0].numpy(), 2)}%.") 

This image most likely belongs to  with a 47.66%.


In [79]:
import os
from PIL import Image 
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError 

In [17]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

path = Path(data_dir).rglob("*\*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

In [97]:
path = Path(data_dir).rglob("*\*.jpg")
for img_p in path:
    try:
        img = Image.open(img_p).convert('RGB')
        img.resize((32, 32), Image.ANTIALIAS)
        # print(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

C:\Users\vllla\AppData\Local\Temp\ipykernel_10156\699106473.py:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img.resize((32, 32), Image.ANTIALIAS)
